In [1]:
#using pyspark 

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
#Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-31 22:07:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-31 22:07:36 (9.59 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import pandas as pd

In [5]:
#get data from S3 Buckets
from pyspark import SparkFiles

In [6]:
# Inflation
url = "https://team5-stock-index-bucket.s3.us-east-2.amazonaws.com/final+project+data/Inflation.csv"
spark.sparkContext.addFile(url)
inflation_df = spark.read.csv(SparkFiles.get("Inflation.csv"), sep=",", header=True, inferSchema=True)

In [7]:
inflation_df.show()

+--------+---------+-------+-------+---------+-------+--------+----------+
|LOCATION|INDICATOR|SUBJECT|MEASURE|FREQUENCY|   TIME|   Value|Flag Codes|
+--------+---------+-------+-------+---------+-------+--------+----------+
|     USA|      CPI|    TOT|IDX2015|        M| 1955-1|11.26501|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-2|11.26501|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-3|11.26501|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-4|11.26501|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-5|11.26501|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-6|11.26501|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-7|11.30721|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-8|11.30721|      null|
|     USA|      CPI|    TOT|IDX2015|        M| 1955-9| 11.3494|      null|
|     USA|      CPI|    TOT|IDX2015|        M|1955-10| 11.3494|      null|
|     USA|      CPI|    T

In [8]:
inflation_df = inflation_df.withColumnRenamed("Flag Codes", "flag_codes")

In [9]:
url = "https://team5-stock-index-bucket.s3.us-east-2.amazonaws.com/final+project+data/NYA.csv"
spark.sparkContext.addFile(url)
NYA_df = spark.read.csv(SparkFiles.get("NYA.csv"), sep=",", header=True, inferSchema=True)

In [10]:
NYA_df.show()

+-----+--------+-------+-------+----------+----------+----------+----------+----------+------+----------+----+
|Index|    Date|  Month|Quarter|      Open|      High|       Low|     Close| Adj Close|Volume|  CloseUSD|_c11|
+-----+--------+-------+-------+----------+----------+----------+----------+----------+------+----------+----+
|  NYA|12/31/65|1965-12|1965-Q4|528.690002|528.690002|528.690002|528.690002|528.690002|     0|528.690002|null|
|  NYA|  1/3/66| 1966-1|1966-Q1|527.210022|527.210022|527.210022|527.210022|527.210022|     0|527.210022|null|
|  NYA|  1/4/66| 1966-1|1966-Q1|527.840027|527.840027|527.840027|527.840027|527.840027|     0|527.840027|null|
|  NYA|  1/5/66| 1966-1|1966-Q1|531.119995|531.119995|531.119995|531.119995|531.119995|     0|531.119995|null|
|  NYA|  1/6/66| 1966-1|1966-Q1|532.070007|532.070007|532.070007|532.070007|532.070007|     0|532.070007|null|
|  NYA|  1/7/66| 1966-1|1966-Q1|532.599976|532.599976|532.599976|532.599976|532.599976|     0|532.599976|null|
|

In [11]:
NYA_df = NYA_df.withColumnRenamed("Adj Close", "adj_close")

In [12]:
url = "https://team5-stock-index-bucket.s3.us-east-2.amazonaws.com/final+project+data/Real+GDP.csv"
spark.sparkContext.addFile(url)
real_gdp_df = spark.read.csv(SparkFiles.get("Real+GDP.csv"), sep=",", header=True, inferSchema=True)

In [13]:
real_gdp_df.show()

+----------------+-------+--------+----+
|observation_date|Quarter|   GDPC1| _c3|
+----------------+-------+--------+----+
|      1947-01-01|1947-Q1| 2034.45|null|
|      1947-04-01|1947-Q2|2029.024|null|
|      1947-07-01|1947-Q3|2024.834|null|
|      1947-10-01|1947-Q4|2056.508|null|
|      1948-01-01|1948-Q1|2087.442|null|
|      1948-04-01|1948-Q2|2121.899|null|
|      1948-07-01|1948-Q3|2134.056|null|
|      1948-10-01|1948-Q4| 2136.44|null|
|      1949-01-01|1949-Q1|2107.001|null|
|      1949-04-01|1949-Q2|2099.814|null|
|      1949-07-01|1949-Q3|2121.493|null|
|      1949-10-01|1949-Q4|2103.688|null|
|      1950-01-01|1950-Q1|2186.365|null|
|      1950-04-01|1950-Q2|2253.045|null|
|      1950-07-01|1950-Q3|2340.112|null|
|      1950-10-01|1950-Q4| 2384.92|null|
|      1951-01-01|1951-Q1|2417.311|null|
|      1951-04-01|1951-Q2|2459.196|null|
|      1951-07-01|1951-Q3| 2509.88|null|
|      1951-10-01|1951-Q4|2515.408|null|
+----------------+-------+--------+----+
only showing top

In [14]:
url = "https://team5-stock-index-bucket.s3.us-east-2.amazonaws.com/final+project+data/Working+Pop.csv"
spark.sparkContext.addFile(url)
population_df = spark.read.csv(SparkFiles.get("Working+Pop.csv"), sep=",", header=True, inferSchema=True)

In [15]:
population_df.show()

+----------------+-------+---------------+
|observation_date|  Month|LFWA64TTUSM647S|
+----------------+-------+---------------+
|      1965-12-01|1965-12|       7.7178E7|
|      1966-01-01| 1966-1|       7.7178E7|
|      1966-02-01| 1966-2|       7.7178E7|
|      1966-03-01| 1966-3|       7.7178E7|
|      1966-04-01| 1966-4|       7.7178E7|
|      1966-05-01| 1966-5|       7.7178E7|
|      1966-06-01| 1966-6|       7.7178E7|
|      1966-07-01| 1966-7|       7.7178E7|
|      1966-08-01| 1966-8|       7.7178E7|
|      1966-09-01| 1966-9|       7.7178E7|
|      1966-10-01|1966-10|       7.7178E7|
|      1966-11-01|1966-11|       7.7178E7|
|      1966-12-01|1966-12|       7.7178E7|
|      1967-01-01| 1967-1|       7.7178E7|
|      1967-02-01| 1967-2|       7.7178E7|
|      1967-03-01| 1967-3|       7.7178E7|
|      1967-04-01| 1967-4|       7.7178E7|
|      1967-05-01| 1967-5|       7.7178E7|
|      1967-06-01| 1967-6|       7.7178E7|
|      1967-07-01| 1967-7|       7.7178E7|
+----------

In [16]:
#checking data types of each df 
print(inflation_df.dtypes)
print(NYA_df.dtypes)
print(real_gdp_df.dtypes)
print(population_df.dtypes)

[('LOCATION', 'string'), ('INDICATOR', 'string'), ('SUBJECT', 'string'), ('MEASURE', 'string'), ('FREQUENCY', 'string'), ('TIME', 'string'), ('Value', 'double'), ('flag_codes', 'string')]
[('Index', 'string'), ('Date', 'string'), ('Month', 'string'), ('Quarter', 'string'), ('Open', 'double'), ('High', 'double'), ('Low', 'double'), ('Close', 'double'), ('adj_close', 'double'), ('Volume', 'bigint'), ('CloseUSD', 'double'), ('_c11', 'string')]
[('observation_date', 'string'), ('Quarter', 'string'), ('GDPC1', 'double'), ('_c3', 'string')]
[('observation_date', 'string'), ('Month', 'string'), ('LFWA64TTUSM647S', 'double')]


In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('finalproject99')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://team5-stock-index-analysis.c5eoj6us86ug.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [ ]:
#write real_gdp_df to realGDP table in RDS
real_gdp_df.write.jdbc(url=jdbc_url, table='realGDP', mode=mode, properties=config)

In [ ]:
#write inflation_df to inflation table in RDS 
inflation_df.write.jdbc(url=jdbc_url, table='inflation', mode=mode, properties=config)

In [ ]:
#write NYA_df to NYA table in RDS 
NYA_df.write.jdbc(url=jdbc_url, table='NYA', mode=mode, properties=config)

In [ ]:
#write population_df to population table in RDS 
population_df.write.jdbc(url=jdbc_url, table='population', mode=mode, properties=config)